In [9]:
import software.math.tsd.webmvp.io.Entry
import software.math.tsd.webmvp.io.loadContent
import software.math.tsd.webmvp.dorep.jekyll.newJekyllBuildingEnvironment
import software.math.tsd.webmvp.io.newEntryFromAbsPath
import software.math.tsd.webmvp.io.path
import software.math.tsd.webmvp.md.plainTextToMarkdownContent
import kotlin.io.path.Path
import kotlin.io.path.absolute
import arrow.core.Either
import arrow.core.getOrElse
import arrow.core.left
import arrow.core.raise.either
import software.math.tsd.webmvp.md.MarkdownContent
import system.file.copyDirectory
import system.file.deleteDirectory
import java.io.IOException
import java.nio.file.Files
import java.nio.file.Path
import kotlin.io.path.exists
import kotlin.io.path.writeText
import arrow.core.right
import software.math.tsd.webmvp.io.id
import software.math.tsd.webmvp.io.name
import software.math.tsd.webmvp.dorep.jekyll.saveNavigation
import software.math.tsd.webmvp.dorep.jekyll.saveToc
import kotlin.io.path.createDirectory
import kotlin.io.path.deleteIfExists
import kotlin.io.path.isDirectory
import kotlin.io.path.isRegularFile
import arrow.core.flatMap
import software.math.tsd.webmvp.dorep.jekyll.build
import software.math.tsd.webmvp.dorep.jekyll.install
import software.math.tsd.webmvp.dorep.jekyll.newTempJekyllDoRepInstaller
import arrow.core.None
import arrow.core.Some
import software.math.tsd.webmvp.dorep.jekyll.FrontMatter
import software.math.tsd.webmvp.dorep.jekyll.JekyllIndex
import software.math.tsd.webmvp.dorep.jekyll.extractAbstract
import software.math.tsd.webmvp.dorep.jekyll.extractTitle
import software.math.tsd.webmvp.dorep.jekyll.generateNav
import software.math.tsd.webmvp.dorep.jekyll.generateToC
import software.math.tsd.webmvp.dorep.jekyll.parse
import software.math.tsd.webmvp.dorep.jekyll.toMarkdownString
import software.math.tsd.webmvp.fromLowercaseToTitleCase
import software.math.tsd.webmvp.io.title
import kotlin.io.path.name
import software.math.tsd.webmvp.dorep.jekyll.copyIndex
import kotlin.io.path.name
import kotlin.io.path.nameWithoutExtension



# TXT to Web

The `txt-to-web` flow builds the static site of a plain text article via DoRep
for Jekyll.

## Sample Blog

In [10]:
val blogRoot = Path("tsd-blog")
val articleRelPath = Path("abstract-text.txt")
val dstPath = Files
    .createTempDirectory("dorep-for-jekyll---web---tsd-web---mvp")

## Article Loading

Load the text article and convert it to a trivial Markdown content to use as
the source for the Jekyll static site generation.

In [11]:
val entry = Entry(blogRoot, articleRelPath)
val content = loadContent(entry).getOrNull()
val markdownContent = content?.plainTextToMarkdownContent()

markdownContent

# Abstract Text

The article "Abstract Text" is a paragraph built into a webpage by Texsydo
Web, where the article source comes from a plain text file.


## Building Intermediate Jekyll Markdown

Texsydo Markdown needs conversion to Jekyll Markdown, which is the input for .

In [12]:
val home = Entry(blogRoot).title.fromLowercaseToTitleCase()
val permalink = entry.path.nameWithoutExtension
val title = markdownContent.extractTitle()
val abstract = markdownContent.extractAbstract()
val jekyllContent = JekyllIndex(
    FrontMatter(
        permalink,
        title,
        Some(abstract),
        ogimage = None,
    ),
    markdownContent.generateNav(home),
    markdownContent.generateToC(),
    markdownContent.parse(),
    None,
)

jekyllContent.toMarkdownString()

---
permalink: abstract-text
title: "Abstract Text"
description: "The article \"Abstract Text\" is a paragraph built into a webpage by Texsydo Web, where the article source comes from a plain text file."
---


# Abstract Text

The article "Abstract Text" is a paragraph built into a webpage by Texsydo
Web, where the article source comes from a plain text file.




## Creating a Building Environment

In [13]:
import software.math.tsd.webmvp.dorep.jekyll.copyJekyllMetaDirectories

val jekyllBuild = newJekyllBuildingEnvironment(blogRoot.toAbsolutePath())
val indexPath = jekyllBuild
    .copyIndex(entry, jekyllContent)
    .getOrNull()

jekyllBuild.copyJekyllMetaDirectories()
    .onLeft { println(it) }

jekyllBuild.saveNavigation(jekyllContent)
    .onLeft { println(it) }

jekyllBuild.saveToc(jekyllContent)
    .onLeft { println(it) }

println("Jekyll environment")
println("Source: file:///${jekyllBuild.src}")
println("Destination: file:///${jekyllBuild.dst}")
println("Markdown index copied to file:///$indexPath")

Jekyll environment
Source: file:////home/tobi/texsydo/texsydo---mvp/tsd-web---mvp/web/docs/integration/tsd-blog
Destination: file:////tmp/build---dorep-for-jekyll---web---tsd-web---mvp4656309354280771367
Markdown index copied to file:////tmp/build---dorep-for-jekyll---web---tsd-web---mvp4656309354280771367/abstract-text/index.md


## Installing DoRep for Jekyll and Building the Static Website

In [14]:
fun build_site() = either {
    val cli = newTempJekyllDoRepInstaller()
        .bind()
        .install()
        .bind()

    println("DoRep for Jekyll installed at file:///${cli.binPath}")

    println("Building site:")
    println("- Source file:///${jekyllBuild.src}")
    println("- From intermediate env file:///${jekyllBuild.dst}")
    println("- To destination file:///$dstPath")

    cli.build(jekyllBuild.dst, dstPath).bind()

    println("Article built at file:///$dstPath")
}

build_site().onLeft { println(it) }

⚙ Clone DoRep for Jekyll
🛈 Repo: https://github.com/texsydo/dorep-for-jekyll.git
🛈 Tag: Option.Some(v0.1.0)
✔ Clone DoRep for Jekyll
⚙ Checkout Tag `v0.1.0`
✔ Checkout Tag `v0.1.0`
⚙ Build for Jekyll
🛈 > Task :checkKotlinGradlePluginConfigurationErrors SKIPPED
> Task :processResources NO-SOURCE

> Task :compileKotlin
w: file:///tmp/dorep-for-jekyll-ops1569844419392128572/dorep-for-jekyll/src/main/kotlin/software/math/tsd/jekyll/Md.kt:56:5 'when' is exhaustive so 'else' is redundant here.

> Task :compileJava NO-SOURCE
> Task :classes UP-TO-DATE
> Task :jar
> Task :startScripts
> Task :installDist

BUILD SUCCESSFUL in 2s
4 actionable tasks: 4 executed
Consider enabling configuration cache to speed up this build: https://docs.gradle.org/9.0.0/userguide/configuration_cache_enabling.html

✔ Build DoRep for Jekyll
DoRep for Jekyll installed at file:////tmp/dorep-for-jekyll-ops1569844419392128572/dorep-for-jekyll/build/install/dorep-for-jekyll/bin
Building site:
- Source file:////home/tobi/t

Either.Right(kotlin.Unit)

### Running the Website

Run an HTTP server (e.g. `http-serve`) in the destination directory, and open
 the path `/abstract-text` (e.g., `http://192.168.0.6:8080/abstract-text`) to
  view the plain text file built as a web
 page
 with MSW Engineer styles and data.